# Enhance Gemini with access to external services with function calling: Challenge Lab

[Lab link](https://partner.cloudskillsboost.google/paths/2294/course_templates/1299/labs/532460)

### Objective

This lab tests your ability to implement the ReAct pattern by extending LLMs to access external information, such as databases, third party services and a coding environment. This lab uses the following technologies and Google Cloud services:
 - Vertex AI
 - Vertex AI Workbench
 - Gemini
 - Function Calling


### Task 1. Open a Jupyter notebook in Vertex AI Workbench

In [ ]:
# install the required libraries:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform==1.88.0

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


In [ ]:
# define project variables and initialize the Vertex AI SDK:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# import the required libraries:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

### Task 2. Define functions in Python

In [ ]:
# TODO: Define a function to reverse the order
# of a string and return the result.
# Keep the print statement within the function.
def reverse(s: str) -> str:
    """
    Reverses the order of a string using slicing.

    Args:
        s: The input string.

    Returns:
        The reversed string.
    """

    print("Calling reverse function")

    return s[::-1]

# Example Usage:
print(f"Original: 'hello', Reversed: '{reverse('hello')}'")
print(f"Original: 'Python', Reversed: '{reverse('Python')}'")
print(f"Original: 'level', Reversed: '{reverse('level')}'")

Calling reverse function
Original: 'hello', Reversed: 'olleh'
Calling reverse function
Original: 'Python', Reversed: 'nohtyP'
Calling reverse function
Original: 'level', Reversed: 'level'


In [2]:
# TODO: Define a function to remove white space
# characters from a string and return the result.
# Keep the print statement within the function.
def remove_white_spaces(s: str) -> str:
    """
    Removes all whitespace characters from a string.

    Args:
        s: The input string.

    Returns:
        The string with all whitespace characters removed.
    """
    # Using split() without arguments splits by any whitespace and
    # discards empty strings, then join() puts the non-whitespace parts back together.
    cleaned_string = "".join(s.split())

    print("Calling remove_white_spaces function")

    return cleaned_string

# Example Usage:
print("\n--- Example 1 ---")
str1 = "  Hello   World!  "
print(f"Original: '{str1}', Without Whitespace: '{remove_white_spaces(str1)}'")

print("\n--- Example 2 ---")
str2 = "  This\tis a\nmulti-line  string with   various\twhitespace.  "
print(f"Original: '{str2}', Without Whitespace: '{remove_white_spaces(str2)}'")

print("\n--- Example 3 ---")
str3 = "NoWhitespaceHere"
print(f"Original: '{str3}', Without Whitespace: '{remove_white_spaces(str3)}'")


--- Example 1 ---
Calling remove_white_spaces function
Original: '  Hello   World!  ', Without Whitespace: 'HelloWorld!'

--- Example 2 ---
Calling remove_white_spaces function
Original: '  This	is a
multi-line  string with   various	whitespace.  ', Without Whitespace: 'Thisisamulti-linestringwithvariouswhitespace.'

--- Example 3 ---
Calling remove_white_spaces function
Original: 'NoWhitespaceHere', Without Whitespace: 'NoWhitespaceHere'


In [ ]:
# Create FunctionDeclarations for my functions:
function_declarations_fd = FunctionDeclaration(
    name="function_declarations",
    description="Reverses the order of characters in a given string and returns the reversed string.",
    parameters={
        "type": "object",
        "properties": {
            "reverse": {
                "type": "string",
                "description": "The input string to be reversed"
            },
            "remove_white_spaces": {
                "type": "string",
                "description": "Removes all whitespace characters (spaces, tabs, newlines, etc.) from a string and returns the cleaned string."
            }
        },
        "required": [
            "reverse",
            "remove_white_spaces"
        ]
    },
)

In [ ]:
# Define a tool with the reverse and remove_white_spaces functions
my_tool = Tool(function_declarations=[function_declarations_fd])

In [ ]:
# Define the system instructions for the model:
system_instruction = """
    - Fulfill the user's instructions, including telling jokes and random facts.
    - If asked to reverse a string or remove whitespace, call the provided functions.
    - You may call one function after the other if needed.
    - Repeat the result to the user.
    """

In [ ]:
# Define and initialize a model with gemini-2.0-flash-001, the tools, a temperature of 0, and system instructions:
model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    generation_config=GenerationConfig(temperature=0),
    system_instruction=system_instruction,
    tools=[my_tool]
)

In [ ]:
# Start a chat session with the model.
chat = model.start_chat()

In [ ]:
# Define a function to handle the response from the model.

def handle_response(response):

    # If there is a function call then invoke it
    # Otherwise print the response.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return
    
    # IF the function_call requests your reverse function
    # Extract the arguments to use in your function
    # Call your function
    # Send the result back to the chat session with the model
    # Recursive call
    if function_call.name == "reverse":
        reverse(function_call.args["my-parameter"])
    
    # the function_call requests your remove_white_spaces function
    # Extract the arguments to use in your function
    # Call your function
    # Send the result back to the chat session with the model
    # Make a recursive call of this handler function
    elif function_call.name == "remove_white_spaces":
        remove_white_spaces(function_call.args["my-parameter"])
    
    else:
        # You shouldn't end up here
        print(function_call)